# Imports and Setups

In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00


In [2]:
! pip install nlpaug==1.1.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 5.3 MB/s eta 0:00:00


In [3]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [4]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=122ab9afbe7bf87755b24f9835934b51f3d28dfabe84844712ffdcf77163328c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
! git clone https://github.com/nng555/ssmba

Cloning into 'ssmba'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 74 (delta 29), reused 60 (delta 15), pack-reused 0
Receiving objects: 100% (74/74), 132.41 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (29/29), done.


Lendo o Arquivo

In [6]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [7]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [8]:
num_gen = 5
perc = 5

# Emoint

In [9]:
import re
import json
# from sklearn.model_selection import train_test_split



def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json['texto']:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "\n"
    f.write(data)




# build_text_files(base_cru_1p,'train_dataset.txt')

In [10]:
def build_label_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json['indices']:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "\n"
    f.write(data)




# build_label_files(base_cru_1p,'label_dataset.txt')

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 8 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/ssmba/base_emoint_'+str(perc)+'_ssmba8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-26 23:58:34.630662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-26 23:58:34.630782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-6-9e2c65708440>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-6-9e2c65708440>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-26 23:59:34.560315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-26 23:59:34.560432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-6-9e2c65708440>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-6-9e2c65708440>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:00:31.651522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:00:31.651656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-6-9e2c65708440>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-6-9e2c65708440>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:01:27.651717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:01:27.651830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-6-9e2c65708440>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-6-9e2c65708440>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:02:24.226656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:02:24.226775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-6-9e2c65708440>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-6-9e2c65708440>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


# TREC

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/trec/base_trec_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 8 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/trec/ssmba/base_trec_'+str(perc)+'_ssmba8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:03:19.862300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:03:19.862403: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-7-ad1f17e37223>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-7-ad1f17e37223>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:04:37.397109: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:04:37.397231: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-7-ad1f17e37223>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-7-ad1f17e37223>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:05:57.074760: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:05:57.074888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-7-ad1f17e37223>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-7-ad1f17e37223>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:07:15.051508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:07:15.051636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-7-ad1f17e37223>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-7-ad1f17e37223>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:08:32.557359: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:08:32.557467: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-7-ad1f17e37223>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-7-ad1f17e37223>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


# Offensive

In [11]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/ssmba/base_offensive_'+str(perc)+'_ssmba'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

2023-09-13 02:33:29.595979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-11-53e8a0e24377>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-11-53e8a0e24377>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:34:34.338926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-11-53e8a0e24377>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-11-53e8a0e24377>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:35:19.391953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-11-53e8a0e24377>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-11-53e8a0e24377>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:36:04.853032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-11-53e8a0e24377>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-11-53e8a0e24377>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:36:48.684056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-11-53e8a0e24377>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-11-53e8a0e24377>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


# Ironic

In [12]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

2023-09-13 02:37:33.338334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-12-256379d5b8f5>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-12-256379d5b8f5>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:38:08.720796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-12-256379d5b8f5>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-12-256379d5b8f5>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:38:43.766921: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-12-256379d5b8f5>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-12-256379d5b8f5>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:39:20.223387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-12-256379d5b8f5>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-12-256379d5b8f5>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:39:56.346112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-12-256379d5b8f5>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-12-256379d5b8f5>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


# Tweet_Sentiment

In [13]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

2023-09-13 02:40:32.041238: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-13-cb8dce7cab25>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-13-cb8dce7cab25>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:41:19.791095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-13-cb8dce7cab25>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-13-cb8dce7cab25>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:42:07.438519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-13-cb8dce7cab25>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-13-cb8dce7cab25>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:42:55.559243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-13-cb8dce7cab25>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-13-cb8dce7cab25>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


2023-09-13 02:43:44.675204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-ar

<ipython-input-13-cb8dce7cab25>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-13-cb8dce7cab25>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


# Emotions

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/base_emotions_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 8 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/ssmba/base_emotions_'+str(perc)+'_ssmba8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:09:52.149037: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:09:52.149154: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-8-867536991c84>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-8-867536991c84>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:13:51.377399: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:13:51.377525: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-8-867536991c84>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-8-867536991c84>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:17:54.581262: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:17:54.582044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-8-867536991c84>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-8-867536991c84>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:21:55.910972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:21:55.911098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

<ipython-input-8-867536991c84>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
<ipython-input-8-867536991c84>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-27 00:25:49.860881: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 00:25:49.861039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

# Rotten

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/rotten/base_rotten_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/rotten/ssmba/base_rotten_'+str(perc)+'_ssmba'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

2023-03-05 00:17:19.633499: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 00:17:20.692061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-05 00:17:20.692185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-05 00:17:20.692208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# SST

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/sst/base_sst_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

  build_text_files(base_cru_1p,'train_dataset.txt')
  build_label_files(base_cru_1p,'label_dataset.txt')

  ! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

  prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['generated'])
  label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])
  fim = pd.concat([prafraseador, label], axis=1)
  export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
  export = export.drop('indices', axis= 1)

  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/sst/ssmba/base_sst_'+str(perc)+'_ssmba'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

2023-03-05 00:24:03.957043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 00:24:04.781395: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-05 00:24:04.781499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-05 00:24:04.781519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF